In [ ]:
import socket
import _thread
import time


def threaded(conn, addr, group):
    print('Connect by',addr)
    while True:
        try:
            data = conn.recv(1024)
            if not data:
                print('Disconnected by',addr)
                group.remove(conn)
                break

            file_path = data.decode() # 첫 번째 수신은 파일명
            print('Receieve File :',file_path)

            data_transferred = 0
            data = conn.recv(1024)
            with open(file_path, "wb") as f:
                try:
                    while data:
                        f.write(data)
                        data_transferred += len(data)
                        data = conn.recv(1024)
                        if str(data).find("Complete") != -1: # 파일 다 받으면 break
                            break
                except Exception as ex:
                    print(ex)

            print("File is saved [byte:"+str(data_transferred)+"]")

            txt = "스팸입니다.\n"
            sendData = txt
            for c in group:
                if c is conn:
                    c.sendall(bytes(sendData,'UTF-8')) # 받은 데이터 파일을 보낸 Client에게만 전송
                    print('Send Data!')
        
        except:
            # 클라이언트 소켓 강제 종료 시 (ex : 네트워크 변경)
            print('예외발생')
            print('Disconnected by',addr)
            group.remove(conn)
            break
    conn.close()
    
host = ''
port = 50000

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((host, port))
server_socket.listen()
print("Listening")

group = []
while True:
    conn, addr = server_socket.accept()
    group.append(conn)
    _thread.start_new_thread(threaded, (conn, addr, group))
    
server_socket.close()
print('서버 종료')

Listening
Connect by ('192.168.0.37', 8307)
Receieve File : 통화 녹음 인턴 황재현_211001_113249.m4a
File is saved [byte:57964]
Send Data!
Disconnected by ('192.168.0.37', 8307)
